# World Bank Data Analysis: Africa GDP Growth and Employment

**Research Question:** How do GDP growth and employment to population ratio change over time in Africa?

**Team Members:**
- [Member 1]
- [Member 2]
- [Member 3]
- [Member 4]
- [Member 5]

**Date:** December 7, 2024

---

## Table of Contents
1. [Setup and Data Collection](#1-setup-and-data-collection)
2. [SQL Database Creation and Data Cleaning](#2-sql-database-creation-and-data-cleaning)
3. [SQL Descriptive Statistics](#3-sql-descriptive-statistics)
4. [Python Data Analysis](#4-python-data-analysis)
5. [Visualizations](#5-visualizations)
6. [Statistical Modeling](#6-statistical-modeling)
7. [Conclusions](#7-conclusions)

---
## 1. Setup and Data Collection

### 1.1 Import Libraries

In [ ]:
# Data fetching and manipulation
import wbgapi as wb
import pandas as pd
import numpy as np

# Database
import sqlite3

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Statistics
from scipy import stats
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Utilities
import os
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("All libraries imported successfully!")

### 1.2 Define World Bank Indicators and Countries

In [ ]:
# Define indicators
INDICATORS = {
    'NY.GDP.MKTP.KD.ZG': 'GDP_growth',  # GDP growth (annual %)
    'SL.EMP.TOTL.SP.ZS': 'Employment_to_population_ratio'  # Employment to population ratio, 15+, total (%)
}

# Time period
START_YEAR = 1990
END_YEAR = 2023

print(f"Indicators to fetch:")
for code, name in INDICATORS.items():
    print(f"  - {name} ({code})")
print(f"\nTime period: {START_YEAR} - {END_YEAR}")

### 1.3 Get List of African Countries

In [ ]:
# Method 1: Try to get from World Bank regions
def get_african_countries():
    """Get list of African countries from World Bank API"""
    african_countries = []
    
    try:
        # Get all economies
        for economy in wb.economy.list():
            # Filter for African regions: SSF (Sub-Saharan Africa), MEA (Middle East & North Africa)
            if hasattr(economy, 'region') and economy.region in ['SSF', 'MEA', 'SSA']:
                african_countries.append({
                    'code': economy.id,
                    'name': economy.value,
                    'region': economy.region
                })
    except:
        print("Could not fetch from API, using manual list...")
        # Fallback: Manual list of major African countries
        manual_list = [
            ('DZA', 'Algeria'), ('AGO', 'Angola'), ('BEN', 'Benin'), ('BWA', 'Botswana'),
            ('BFA', 'Burkina Faso'), ('CMR', 'Cameroon'), ('CIV', "Cote d'Ivoire"), 
            ('EGY', 'Egypt'), ('ETH', 'Ethiopia'), ('GHA', 'Ghana'), ('KEN', 'Kenya'),
            ('MAR', 'Morocco'), ('NGA', 'Nigeria'), ('RWA', 'Rwanda'), ('SEN', 'Senegal'),
            ('ZAF', 'South Africa'), ('TZA', 'Tanzania'), ('UGA', 'Uganda'), 
            ('ZMB', 'Zambia'), ('ZWE', 'Zimbabwe')
        ]
        african_countries = [{'code': code, 'name': name, 'region': 'SSF'} 
                           for code, name in manual_list]
    
    return pd.DataFrame(african_countries)

# Get African countries
african_countries_df = get_african_countries()
print(f"Found {len(african_countries_df)} African countries/economies")
print(f"\nFirst 10 countries:")
print(african_countries_df.head(10))

### 1.4 Fetch Data from World Bank API

In [ ]:
def fetch_indicator_data(indicator_code, countries, start_year=1990, end_year=2023):
    """Fetch data for a specific indicator"""
    try:
        print(f"Fetching {indicator_code}...")
        data = wb.data.DataFrame(
            indicator_code,
            countries,
            time=range(start_year, end_year + 1),
            numericTimeKeys=True,
            columns='series'
        )
        return data
    except Exception as e:
        print(f"Error fetching {indicator_code}: {e}")
        return None

# Fetch data for each indicator
country_codes = african_countries_df['code'].tolist()
all_data = []

for indicator_code, indicator_name in INDICATORS.items():
    print(f"\nFetching {indicator_name}...")
    data = fetch_indicator_data(indicator_code, country_codes, START_YEAR, END_YEAR)
    
    if data is not None:
        # Reshape from wide to long format
        data_long = data.reset_index()
        data_long = data_long.melt(
            id_vars=['economy'],
            var_name='year',
            value_name=indicator_name
        )
        all_data.append(data_long)
        print(f"  ✓ Fetched {len(data_long)} records")

# Combine all indicators
if len(all_data) >= 2:
    df_raw = all_data[0].merge(all_data[1], on=['economy', 'year'], how='outer')
    print(f"\n✓ Combined dataset created: {df_raw.shape[0]} rows, {df_raw.shape[1]} columns")
else:
    print("Error: Could not fetch enough data!")
    df_raw = None

In [ ]:
# Preview raw data
if df_raw is not None:
    print("Raw Data Preview:")
    print(df_raw.head(20))
    print(f"\nData shape: {df_raw.shape}")
    print(f"\nMissing values:")
    print(df_raw.isnull().sum())
    print(f"\nData types:")
    print(df_raw.dtypes)

---
## 2. SQL Database Creation and Data Cleaning

### 2.1 Create SQLite Database and Tables

In [ ]:
# Create data directory if it doesn't exist
os.makedirs('data', exist_ok=True)

# Database path
DB_PATH = 'data/africa_wdi.db'

# Connect to SQLite database (creates if doesn't exist)
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

print(f"Connected to database: {DB_PATH}")

In [ ]:
# Create tables
create_tables_sql = """
-- Drop tables if they exist
DROP TABLE IF EXISTS raw_data;
DROP TABLE IF EXISTS cleaned_data;
DROP TABLE IF EXISTS country_info;

-- Table for raw data from World Bank
CREATE TABLE IF NOT EXISTS raw_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    country_code TEXT NOT NULL,
    country_name TEXT,
    year INTEGER NOT NULL,
    gdp_growth REAL,
    employment_ratio REAL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Table for country information
CREATE TABLE IF NOT EXISTS country_info (
    country_code TEXT PRIMARY KEY,
    country_name TEXT NOT NULL,
    region TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Table for cleaned data
CREATE TABLE IF NOT EXISTS cleaned_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    country_code TEXT NOT NULL,
    country_name TEXT NOT NULL,
    year INTEGER NOT NULL,
    gdp_growth REAL,
    employment_ratio REAL,
    is_outlier_gdp BOOLEAN DEFAULT 0,
    is_outlier_employment BOOLEAN DEFAULT 0,
    data_quality_flag TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    UNIQUE(country_code, year)
);

-- Create indexes
CREATE INDEX IF NOT EXISTS idx_raw_country_year ON raw_data(country_code, year);
CREATE INDEX IF NOT EXISTS idx_cleaned_country_year ON cleaned_data(country_code, year);
CREATE INDEX IF NOT EXISTS idx_year ON cleaned_data(year);
"""

# Execute SQL
conn.executescript(create_tables_sql)
conn.commit()

print("✓ Tables created successfully")

# Verify tables
tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name",
    conn
)
print(f"\nTables in database:")
print(tables)

### 2.2 Load Raw Data into Database

In [ ]:
# Prepare data for database
if df_raw is not None:
    # Rename columns to match database schema
    df_for_db = df_raw.copy()
    df_for_db.columns = ['country_code', 'year', 'gdp_growth', 'employment_ratio']
    
    # Add country names from our country list
    df_for_db = df_for_db.merge(
        african_countries_df[['code', 'name']], 
        left_on='country_code', 
        right_on='code', 
        how='left'
    )
    df_for_db['country_name'] = df_for_db['name'].fillna(df_for_db['country_code'])
    df_for_db = df_for_db[['country_code', 'country_name', 'year', 'gdp_growth', 'employment_ratio']]
    
    # Load into database
    df_for_db.to_sql('raw_data', conn, if_exists='append', index=False)
    
    print(f"✓ Loaded {len(df_for_db)} rows into raw_data table")
    
    # Also load country info
    country_info = african_countries_df[['code', 'name', 'region']].copy()
    country_info.columns = ['country_code', 'country_name', 'region']
    country_info.to_sql('country_info', conn, if_exists='replace', index=False)
    
    print(f"✓ Loaded {len(country_info)} countries into country_info table")
else:
    print("Error: No raw data to load!")

In [ ]:
# Verify data loaded
sample_data = pd.read_sql_query("SELECT * FROM raw_data LIMIT 10", conn)
print("Sample from raw_data table:")
print(sample_data)

### 2.3 Data Cleaning with SQL

In [ ]:
# SQL for data cleaning
cleaning_sql = """
-- Step 1: Remove complete duplicates
DELETE FROM raw_data
WHERE id NOT IN (
    SELECT MIN(id)
    FROM raw_data
    GROUP BY country_code, year
);

-- Step 2: Insert cleaned data with quality flags
INSERT INTO cleaned_data (
    country_code,
    country_name,
    year,
    gdp_growth,
    employment_ratio,
    is_outlier_gdp,
    is_outlier_employment,
    data_quality_flag
)
SELECT
    r.country_code,
    r.country_name,
    r.year,
    r.gdp_growth,
    r.employment_ratio,
    -- Flag GDP outliers (beyond 3 standard deviations)
    CASE
        WHEN r.gdp_growth IS NOT NULL AND
             ABS(r.gdp_growth - (SELECT AVG(gdp_growth) FROM raw_data WHERE gdp_growth IS NOT NULL)) >
             3 * (SELECT
                    SQRT(AVG((gdp_growth - (SELECT AVG(gdp_growth) FROM raw_data WHERE gdp_growth IS NOT NULL)) *
                             (gdp_growth - (SELECT AVG(gdp_growth) FROM raw_data WHERE gdp_growth IS NOT NULL))))
                  FROM raw_data WHERE gdp_growth IS NOT NULL)
        THEN 1
        ELSE 0
    END as is_outlier_gdp,
    -- Flag employment outliers
    CASE
        WHEN r.employment_ratio IS NOT NULL AND
             ABS(r.employment_ratio - (SELECT AVG(employment_ratio) FROM raw_data WHERE employment_ratio IS NOT NULL)) >
             3 * (SELECT
                    SQRT(AVG((employment_ratio - (SELECT AVG(employment_ratio) FROM raw_data WHERE employment_ratio IS NOT NULL)) *
                             (employment_ratio - (SELECT AVG(employment_ratio) FROM raw_data WHERE employment_ratio IS NOT NULL))))
                  FROM raw_data WHERE employment_ratio IS NOT NULL)
        THEN 1
        ELSE 0
    END as is_outlier_employment,
    -- Data quality flag
    CASE
        WHEN r.gdp_growth IS NULL AND r.employment_ratio IS NULL THEN 'MISSING_ALL'
        WHEN r.gdp_growth IS NULL THEN 'MISSING_GDP'
        WHEN r.employment_ratio IS NULL THEN 'MISSING_EMPLOYMENT'
        ELSE 'COMPLETE'
    END as data_quality_flag
FROM raw_data r;
"""

# Execute cleaning
print("Executing data cleaning...")
conn.executescript(cleaning_sql)
conn.commit()
print("✓ Data cleaning complete")

In [ ]:
# Check cleaned data
cleaned_count = pd.read_sql_query("SELECT COUNT(*) as count FROM cleaned_data", conn)
print(f"Total records in cleaned_data: {cleaned_count['count'].iloc[0]}")

# Check data quality distribution
quality_dist = pd.read_sql_query(
    "SELECT data_quality_flag, COUNT(*) as count FROM cleaned_data GROUP BY data_quality_flag",
    conn
)
print("\nData Quality Distribution:")
print(quality_dist)

# Check for outliers
outliers = pd.read_sql_query(
    """SELECT 
        SUM(is_outlier_gdp) as gdp_outliers,
        SUM(is_outlier_employment) as employment_outliers
       FROM cleaned_data""",
    conn
)
print("\nOutliers Detected:")
print(outliers)

---
## 3. SQL Descriptive Statistics

### 3.1 Overall Statistics

In [ ]:
# Overall statistics
overall_stats = pd.read_sql_query("""
    SELECT
        COUNT(*) as total_records,
        COUNT(DISTINCT country_code) as num_countries,
        MIN(year) as start_year,
        MAX(year) as end_year,
        COUNT(CASE WHEN gdp_growth IS NOT NULL THEN 1 END) as gdp_records,
        COUNT(CASE WHEN employment_ratio IS NOT NULL THEN 1 END) as employment_records,
        COUNT(CASE WHEN data_quality_flag = 'COMPLETE' THEN 1 END) as complete_records,
        ROUND(100.0 * COUNT(CASE WHEN data_quality_flag = 'COMPLETE' THEN 1 END) / COUNT(*), 2) as completeness_pct
    FROM cleaned_data
""", conn)

print("=" * 60)
print("OVERALL STATISTICS")
print("=" * 60)
print(overall_stats.T)

### 3.2 GDP Growth Statistics

In [ ]:
# GDP statistics
gdp_stats = pd.read_sql_query("""
    SELECT
        ROUND(AVG(gdp_growth), 2) as mean,
        ROUND(MIN(gdp_growth), 2) as min,
        ROUND(MAX(gdp_growth), 2) as max,
        COUNT(gdp_growth) as count,
        COUNT(CASE WHEN is_outlier_gdp = 1 THEN 1 END) as num_outliers
    FROM cleaned_data
    WHERE gdp_growth IS NOT NULL
""", conn)

print("=" * 60)
print("GDP GROWTH STATISTICS")
print("=" * 60)
print(gdp_stats.T)

### 3.3 Employment Ratio Statistics

In [ ]:
# Employment statistics
employment_stats = pd.read_sql_query("""
    SELECT
        ROUND(AVG(employment_ratio), 2) as mean,
        ROUND(MIN(employment_ratio), 2) as min,
        ROUND(MAX(employment_ratio), 2) as max,
        COUNT(employment_ratio) as count,
        COUNT(CASE WHEN is_outlier_employment = 1 THEN 1 END) as num_outliers
    FROM cleaned_data
    WHERE employment_ratio IS NOT NULL
""", conn)

print("=" * 60)
print("EMPLOYMENT RATIO STATISTICS")
print("=" * 60)
print(employment_stats.T)

### 3.4 Statistics by Country

In [ ]:
# Country-level statistics
country_stats = pd.read_sql_query("""
    SELECT
        country_name,
        COUNT(*) as num_years,
        ROUND(AVG(gdp_growth), 2) as avg_gdp_growth,
        ROUND(AVG(employment_ratio), 2) as avg_employment_ratio,
        MIN(year) as first_year,
        MAX(year) as last_year,
        COUNT(CASE WHEN data_quality_flag = 'COMPLETE' THEN 1 END) as complete_years
    FROM cleaned_data
    WHERE gdp_growth IS NOT NULL OR employment_ratio IS NOT NULL
    GROUP BY country_code, country_name
    ORDER BY avg_gdp_growth DESC
""", conn)

print("=" * 60)
print("STATISTICS BY COUNTRY (Top 15 by GDP Growth)")
print("=" * 60)
print(country_stats.head(15))

### 3.5 Statistics by Decade

In [ ]:
# Decade-level statistics
decade_stats = pd.read_sql_query("""
    SELECT
        CASE
            WHEN year BETWEEN 1990 AND 1999 THEN '1990s'
            WHEN year BETWEEN 2000 AND 2009 THEN '2000s'
            WHEN year BETWEEN 2010 AND 2019 THEN '2010s'
            WHEN year BETWEEN 2020 AND 2029 THEN '2020s'
            ELSE 'Other'
        END as decade,
        COUNT(*) as num_records,
        ROUND(AVG(gdp_growth), 2) as avg_gdp_growth,
        ROUND(AVG(employment_ratio), 2) as avg_employment_ratio,
        COUNT(DISTINCT country_code) as num_countries
    FROM cleaned_data
    WHERE year >= 1990 AND (gdp_growth IS NOT NULL OR employment_ratio IS NOT NULL)
    GROUP BY decade
    ORDER BY decade
""", conn)

print("=" * 60)
print("STATISTICS BY DECADE")
print("=" * 60)
print(decade_stats)

### 3.6 Data Completeness by Year

In [ ]:
# Completeness by year
completeness_by_year = pd.read_sql_query("""
    SELECT
        year,
        COUNT(*) as total_records,
        COUNT(CASE WHEN gdp_growth IS NOT NULL THEN 1 END) as gdp_available,
        COUNT(CASE WHEN employment_ratio IS NOT NULL THEN 1 END) as employment_available,
        COUNT(CASE WHEN data_quality_flag = 'COMPLETE' THEN 1 END) as complete_records,
        ROUND(100.0 * COUNT(CASE WHEN data_quality_flag = 'COMPLETE' THEN 1 END) / COUNT(*), 1) as completeness_pct
    FROM cleaned_data
    WHERE year >= 1990
    GROUP BY year
    ORDER BY year DESC
    LIMIT 15
""", conn)

print("=" * 60)
print("DATA COMPLETENESS BY YEAR (Last 15 years)")
print("=" * 60)
print(completeness_by_year)

---
## 4. Python Data Analysis

### 4.1 Load Cleaned Data into Pandas

In [ ]:
# Load cleaned data with complete records only
df = pd.read_sql_query("""
    SELECT * FROM cleaned_data 
    WHERE data_quality_flag = 'COMPLETE'
    ORDER BY country_code, year
""", conn)

print(f"Loaded {len(df)} complete records")
print(f"Countries: {df['country_code'].nunique()}")
print(f"Years: {df['year'].min()} - {df['year'].max()}")
print(f"\nDataset shape: {df.shape}")
print(f"\nFirst few rows:")
df.head(10)

### 4.2 Pandas Descriptive Statistics

In [ ]:
# Summary statistics with pandas
print("Descriptive Statistics:")
df[['gdp_growth', 'employment_ratio']].describe()

In [ ]:
# Additional statistics
print("Additional Statistics:")
print(f"\nGDP Growth:")
print(f"  Median: {df['gdp_growth'].median():.2f}%")
print(f"  Skewness: {df['gdp_growth'].skew():.2f}")
print(f"  Kurtosis: {df['gdp_growth'].kurtosis():.2f}")

print(f"\nEmployment Ratio:")
print(f"  Median: {df['employment_ratio'].median():.2f}%")
print(f"  Skewness: {df['employment_ratio'].skew():.2f}")
print(f"  Kurtosis: {df['employment_ratio'].kurtosis():.2f}")

### 4.3 Time Series Aggregation

In [ ]:
# Aggregate by year
gdp_by_year = df.groupby('year')['gdp_growth'].agg(['mean', 'std', 'median', 'count']).reset_index()
employment_by_year = df.groupby('year')['employment_ratio'].agg(['mean', 'std', 'median', 'count']).reset_index()

print("GDP Growth by Year (Last 10 years):")
print(gdp_by_year.tail(10))

print("\nEmployment Ratio by Year (Last 10 years):")
print(employment_by_year.tail(10))

### 4.4 Correlation Analysis

In [ ]:
# Correlation matrix
correlation_matrix = df[['gdp_growth', 'employment_ratio']].corr()
print("Correlation Matrix:")
print(correlation_matrix)

# Statistical test
pearson_corr, p_value = stats.pearsonr(df['gdp_growth'], df['employment_ratio'])
print(f"\nPearson Correlation Coefficient: {pearson_corr:.4f}")
print(f"P-value: {p_value:.6f}")
print(f"Significance: {'Significant' if p_value < 0.05 else 'Not significant'} at 5% level")

# Spearman correlation (non-parametric)
spearman_corr, spearman_p = stats.spearmanr(df['gdp_growth'], df['employment_ratio'])
print(f"\nSpearman Correlation Coefficient: {spearman_corr:.4f}")
print(f"P-value: {spearman_p:.6f}")

---
## 5. Visualizations

### 5.1 Distribution Plots

In [ ]:
# GDP Growth Distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(df['gdp_growth'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].axvline(df['gdp_growth'].mean(), color='red', linestyle='--', linewidth=2, 
                label=f'Mean: {df["gdp_growth"].mean():.2f}%')
axes[0].axvline(df['gdp_growth'].median(), color='green', linestyle='--', linewidth=2, 
                label=f'Median: {df["gdp_growth"].median():.2f}%')
axes[0].set_xlabel('GDP Growth (%)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of GDP Growth in Africa', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Box plot
axes[1].boxplot(df['gdp_growth'], vert=True, patch_artist=True,
                boxprops=dict(facecolor='steelblue', alpha=0.7))
axes[1].set_ylabel('GDP Growth (%)', fontsize=12)
axes[1].set_title('GDP Growth Box Plot', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Employment Ratio Distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(df['employment_ratio'], bins=50, edgecolor='black', alpha=0.7, color='green')
axes[0].axvline(df['employment_ratio'].mean(), color='red', linestyle='--', linewidth=2,
                label=f'Mean: {df["employment_ratio"].mean():.2f}%')
axes[0].axvline(df['employment_ratio'].median(), color='orange', linestyle='--', linewidth=2,
                label=f'Median: {df["employment_ratio"].median():.2f}%')
axes[0].set_xlabel('Employment to Population Ratio (%)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of Employment Ratio in Africa', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Box plot
axes[1].boxplot(df['employment_ratio'], vert=True, patch_artist=True,
                boxprops=dict(facecolor='green', alpha=0.7))
axes[1].set_ylabel('Employment to Population Ratio (%)', fontsize=12)
axes[1].set_title('Employment Ratio Box Plot', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 5.2 Time Series Plots

In [ ]:
# GDP Growth over time
plt.figure(figsize=(14, 6))
plt.plot(gdp_by_year['year'], gdp_by_year['mean'], linewidth=2.5, marker='o', 
         markersize=5, color='steelblue', label='Mean GDP Growth')
plt.fill_between(gdp_by_year['year'], 
                 gdp_by_year['mean'] - gdp_by_year['std'], 
                 gdp_by_year['mean'] + gdp_by_year['std'], 
                 alpha=0.3, color='steelblue', label='±1 Std Dev')
plt.axhline(y=0, color='red', linestyle='--', alpha=0.5, linewidth=1.5)
plt.axhline(y=gdp_by_year['mean'].mean(), color='green', linestyle=':', 
            alpha=0.7, linewidth=1.5, label=f'Overall Mean: {gdp_by_year["mean"].mean():.2f}%')
plt.xlabel('Year', fontsize=12)
plt.ylabel('GDP Growth (%)', fontsize=12)
plt.title('Average GDP Growth in Africa Over Time', fontsize=14, fontweight='bold')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Employment Ratio over time
plt.figure(figsize=(14, 6))
plt.plot(employment_by_year['year'], employment_by_year['mean'], linewidth=2.5, 
         marker='s', markersize=5, color='green', label='Mean Employment Ratio')
plt.fill_between(employment_by_year['year'], 
                 employment_by_year['mean'] - employment_by_year['std'], 
                 employment_by_year['mean'] + employment_by_year['std'], 
                 alpha=0.3, color='green', label='±1 Std Dev')
plt.axhline(y=employment_by_year['mean'].mean(), color='orange', linestyle=':', 
            alpha=0.7, linewidth=1.5, 
            label=f'Overall Mean: {employment_by_year["mean"].mean():.2f}%')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Employment to Population Ratio (%)', fontsize=12)
plt.title('Average Employment to Population Ratio in Africa Over Time', fontsize=14, fontweight='bold')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Combined time series with dual axes
fig, ax1 = plt.subplots(figsize=(14, 6))

# GDP Growth on left y-axis
color = 'steelblue'
ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('GDP Growth (%)', color=color, fontsize=12)
line1 = ax1.plot(gdp_by_year['year'], gdp_by_year['mean'], color=color, linewidth=2.5, 
                 marker='o', markersize=5, label='GDP Growth')
ax1.tick_params(axis='y', labelcolor=color)
ax1.axhline(y=0, color='red', linestyle='--', alpha=0.3, linewidth=1)
ax1.grid(alpha=0.3)

# Employment Ratio on right y-axis
ax2 = ax1.twinx()
color = 'green'
ax2.set_ylabel('Employment to Population Ratio (%)', color=color, fontsize=12)
line2 = ax2.plot(employment_by_year['year'], employment_by_year['mean'], color=color, 
                 linewidth=2.5, marker='s', markersize=5, label='Employment Ratio')
ax2.tick_params(axis='y', labelcolor=color)

# Combined legend
lines = line1 + line2
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='upper left')

plt.title('GDP Growth and Employment Ratio in Africa Over Time', fontsize=14, fontweight='bold')
fig.tight_layout()
plt.show()

### 5.3 Correlation and Scatter Plots

In [ ]:
# Scatter plot with regression line
plt.figure(figsize=(12, 7))
plt.scatter(df['gdp_growth'], df['employment_ratio'], alpha=0.5, s=30, color='steelblue')
plt.xlabel('GDP Growth (%)', fontsize=12)
plt.ylabel('Employment to Population Ratio (%)', fontsize=12)
plt.title('GDP Growth vs Employment Ratio in Africa', fontsize=14, fontweight='bold')

# Add regression line
z = np.polyfit(df['gdp_growth'], df['employment_ratio'], 1)
p = np.poly1d(z)
x_line = np.linspace(df['gdp_growth'].min(), df['gdp_growth'].max(), 100)
plt.plot(x_line, p(x_line), "r--", linewidth=2.5, 
         label=f'Regression Line: y = {z[0]:.3f}x + {z[1]:.2f}')

# Add correlation coefficient
plt.text(0.05, 0.95, f'Pearson r = {pearson_corr:.4f}\np-value = {p_value:.6f}', 
         transform=plt.gca().transAxes, fontsize=11, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Heatmap of correlation
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, 
            fmt='.4f', vmin=-1, vmax=1)
plt.title('Correlation Matrix: GDP Growth vs Employment Ratio', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 5.4 Country Comparison

In [ ]:
# Top 10 countries by average GDP growth
country_avg = df.groupby('country_name').agg({
    'gdp_growth': 'mean',
    'employment_ratio': 'mean',
    'year': 'count'
}).reset_index()
country_avg.columns = ['country_name', 'avg_gdp_growth', 'avg_employment_ratio', 'num_years']

# Filter countries with at least 15 years of data
country_avg = country_avg[country_avg['num_years'] >= 15]
top_10_gdp = country_avg.nlargest(10, 'avg_gdp_growth')

plt.figure(figsize=(12, 7))
bars = plt.barh(top_10_gdp['country_name'], top_10_gdp['avg_gdp_growth'], color='steelblue')
plt.xlabel('Average GDP Growth (%)', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.title('Top 10 African Countries by Average GDP Growth (1990-2023)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()

# Add value labels
for i, (bar, val) in enumerate(zip(bars, top_10_gdp['avg_gdp_growth'])):
    plt.text(val + 0.1, bar.get_y() + bar.get_height()/2, f'{val:.2f}%', 
             va='center', fontsize=10)

plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Interactive scatter plot with Plotly
fig = px.scatter(country_avg, 
                 x='avg_gdp_growth', 
                 y='avg_employment_ratio',
                 size='num_years',
                 hover_name='country_name',
                 hover_data={'avg_gdp_growth': ':.2f',
                            'avg_employment_ratio': ':.2f',
                            'num_years': True},
                 labels={
                     'avg_gdp_growth': 'Average GDP Growth (%)',
                     'avg_employment_ratio': 'Average Employment Ratio (%)',
                     'num_years': 'Years of Data'
                 },
                 title='African Countries: GDP Growth vs Employment Ratio',
                 color='avg_gdp_growth',
                 color_continuous_scale='RdYlGn')

fig.update_layout(height=600, width=900)
fig.show()

### 5.5 Decade Comparison

In [ ]:
# Add decade column
df['decade'] = (df['year'] // 10) * 10
df['decade_label'] = df['decade'].astype(str) + 's'

# Group by decade
decade_avg = df[df['year'] >= 1990].groupby('decade_label').agg({
    'gdp_growth': 'mean',
    'employment_ratio': 'mean'
}).reset_index()

# Create grouped bar chart
x = np.arange(len(decade_avg))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
bars1 = ax.bar(x - width/2, decade_avg['gdp_growth'], width, label='GDP Growth', color='steelblue')
bars2 = ax.bar(x + width/2, decade_avg['employment_ratio']/10, width, 
               label='Employment Ratio (÷10)', color='green')

ax.set_xlabel('Decade', fontsize=12)
ax.set_ylabel('Value (%)', fontsize=12)
ax.set_title('GDP Growth and Employment Ratio by Decade', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(decade_avg['decade_label'])
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Add value labels
for bar in bars1:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

---
## 6. Statistical Modeling

### 6.1 Linear Regression Model

In [ ]:
# Prepare data for regression
X = df['gdp_growth'].values
y = df['employment_ratio'].values

# Add constant for intercept
X_with_const = sm.add_constant(X)

# Fit OLS model
model = sm.OLS(y, X_with_const).fit()

# Print summary
print("=" * 80)
print("LINEAR REGRESSION MODEL: Employment Ratio ~ GDP Growth")
print("=" * 80)
print(model.summary())

In [ ]:
# Extract key metrics
print("\nKey Regression Metrics:")
print(f"  R-squared: {model.rsquared:.4f}")
print(f"  Adjusted R-squared: {model.rsquared_adj:.4f}")
print(f"  F-statistic: {model.fvalue:.4f}")
print(f"  Prob (F-statistic): {model.f_pvalue:.6f}")
print(f"\nCoefficients:")
print(f"  Intercept: {model.params[0]:.4f}")
print(f"  GDP Growth coefficient: {model.params[1]:.4f}")
print(f"\nInterpretation:")
print(f"  A 1% increase in GDP growth is associated with a {model.params[1]:.4f}% change in employment ratio")

### 6.2 Residual Analysis

In [ ]:
# Plot residuals
residuals = model.resid
fitted_values = model.fittedvalues

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Residuals vs Fitted
axes[0, 0].scatter(fitted_values, residuals, alpha=0.5)
axes[0, 0].axhline(y=0, color='r', linestyle='--')
axes[0, 0].set_xlabel('Fitted Values')
axes[0, 0].set_ylabel('Residuals')
axes[0, 0].set_title('Residuals vs Fitted Values')
axes[0, 0].grid(alpha=0.3)

# 2. Q-Q Plot
stats.probplot(residuals, dist="norm", plot=axes[0, 1])
axes[0, 1].set_title('Normal Q-Q Plot')
axes[0, 1].grid(alpha=0.3)

# 3. Scale-Location
axes[1, 0].scatter(fitted_values, np.sqrt(np.abs(residuals)), alpha=0.5)
axes[1, 0].set_xlabel('Fitted Values')
axes[1, 0].set_ylabel('√|Residuals|')
axes[1, 0].set_title('Scale-Location Plot')
axes[1, 0].grid(alpha=0.3)

# 4. Histogram of Residuals
axes[1, 1].hist(residuals, bins=30, edgecolor='black', alpha=0.7)
axes[1, 1].set_xlabel('Residuals')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Distribution of Residuals')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 6.3 Normality Tests

In [ ]:
# Shapiro-Wilk test for normality
shapiro_stat, shapiro_p = stats.shapiro(residuals)
print("Shapiro-Wilk Test for Normality of Residuals:")
print(f"  Test Statistic: {shapiro_stat:.4f}")
print(f"  P-value: {shapiro_p:.6f}")
print(f"  Conclusion: Residuals are {'NOT ' if shapiro_p < 0.05 else ''}normally distributed (α=0.05)")

# Jarque-Bera test
jb_stat, jb_p = stats.jarque_bera(residuals)
print(f"\nJarque-Bera Test:")
print(f"  Test Statistic: {jb_stat:.4f}")
print(f"  P-value: {jb_p:.6f}")

### 6.4 Time Series Analysis by Country

In [ ]:
# Select a few countries for detailed time series analysis
selected_countries = country_avg.nlargest(5, 'num_years')['country_name'].tolist()
print(f"Analyzing time series for: {selected_countries}")

# Plot time series for selected countries
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

for country in selected_countries:
    country_data = df[df['country_name'] == country].sort_values('year')
    axes[0].plot(country_data['year'], country_data['gdp_growth'], 
                marker='o', label=country, linewidth=2)
    axes[1].plot(country_data['year'], country_data['employment_ratio'], 
                marker='s', label=country, linewidth=2)

axes[0].set_xlabel('Year', fontsize=12)
axes[0].set_ylabel('GDP Growth (%)', fontsize=12)
axes[0].set_title('GDP Growth Over Time by Country', fontsize=14, fontweight='bold')
axes[0].legend(loc='best')
axes[0].grid(alpha=0.3)
axes[0].axhline(y=0, color='red', linestyle='--', alpha=0.3)

axes[1].set_xlabel('Year', fontsize=12)
axes[1].set_ylabel('Employment Ratio (%)', fontsize=12)
axes[1].set_title('Employment to Population Ratio Over Time by Country', fontsize=14, fontweight='bold')
axes[1].legend(loc='best')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

---
## 7. Conclusions

### 7.1 Summary of Findings

In [ ]:
# Generate summary statistics for conclusions
print("=" * 80)
print("SUMMARY OF KEY FINDINGS")
print("=" * 80)

print(f"\n1. DATA COVERAGE:")
print(f"   - Total countries analyzed: {df['country_code'].nunique()}")
print(f"   - Time period: {df['year'].min()} - {df['year'].max()}")
print(f"   - Total observations: {len(df)}")

print(f"\n2. GDP GROWTH:")
print(f"   - Average: {df['gdp_growth'].mean():.2f}%")
print(f"   - Median: {df['gdp_growth'].median():.2f}%")
print(f"   - Range: {df['gdp_growth'].min():.2f}% to {df['gdp_growth'].max():.2f}%")
print(f"   - Standard deviation: {df['gdp_growth'].std():.2f}%")

print(f"\n3. EMPLOYMENT RATIO:")
print(f"   - Average: {df['employment_ratio'].mean():.2f}%")
print(f"   - Median: {df['employment_ratio'].median():.2f}%")
print(f"   - Range: {df['employment_ratio'].min():.2f}% to {df['employment_ratio'].max():.2f}%")
print(f"   - Standard deviation: {df['employment_ratio'].std():.2f}%")

print(f"\n4. CORRELATION:")
print(f"   - Pearson correlation: {pearson_corr:.4f}")
print(f"   - P-value: {p_value:.6f}")
print(f"   - Significance: {'Significant' if p_value < 0.05 else 'Not significant'} at 5% level")
print(f"   - Interpretation: {'Weak positive' if 0 < pearson_corr < 0.3 else 'Moderate positive' if 0.3 <= pearson_corr < 0.7 else 'Strong positive' if pearson_corr >= 0.7 else 'Weak negative' if -0.3 < pearson_corr < 0 else 'Moderate negative' if -0.7 < pearson_corr <= -0.3 else 'Strong negative'} relationship")

print(f"\n5. REGRESSION MODEL:")
print(f"   - R-squared: {model.rsquared:.4f}")
print(f"   - GDP Growth explains {model.rsquared*100:.2f}% of variance in Employment Ratio")
print(f"   - Coefficient: {model.params[1]:.4f}")
print(f"   - Interpretation: 1% increase in GDP growth → {model.params[1]:.4f}% change in employment ratio")

print(f"\n6. TRENDS OVER TIME:")
# Calculate trend using recent years vs earlier years
early_period = df[df['year'] <= 2000]['gdp_growth'].mean()
recent_period = df[df['year'] >= 2010]['gdp_growth'].mean()
print(f"   - GDP Growth (≤2000): {early_period:.2f}%")
print(f"   - GDP Growth (≥2010): {recent_period:.2f}%")
print(f"   - Change: {recent_period - early_period:+.2f}%")

early_emp = df[df['year'] <= 2000]['employment_ratio'].mean()
recent_emp = df[df['year'] >= 2010]['employment_ratio'].mean()
print(f"   - Employment Ratio (≤2000): {early_emp:.2f}%")
print(f"   - Employment Ratio (≥2010): {recent_emp:.2f}%")
print(f"   - Change: {recent_emp - early_emp:+.2f}%")

### 7.2 Answer to Research Question

**Research Question:** How do GDP growth and employment to population ratio change over time in Africa?

**Key Findings:**

1. **GDP Growth Trends:**
   - African countries experienced variable GDP growth over the study period
   - Average GDP growth varies significantly across decades and countries
   - There is considerable volatility in growth rates, reflecting economic shocks and structural changes

2. **Employment Ratio Trends:**
   - Employment to population ratios show relatively stable patterns compared to GDP growth
   - Changes in employment ratios are more gradual than GDP fluctuations
   - Significant variation exists across countries

3. **Relationship Between GDP and Employment:**
   - Statistical analysis reveals the nature and strength of the relationship
   - The correlation coefficient indicates the degree of association
   - Regression analysis quantifies how changes in GDP growth relate to employment ratios

4. **Country-Level Variations:**
   - Different African countries show distinct patterns
   - Top performers demonstrate consistent growth and employment trends
   - Economic structure and policies influence the GDP-employment relationship

5. **Temporal Patterns:**
   - Decade-by-decade analysis reveals evolving trends
   - Recent years show different patterns compared to earlier periods
   - External shocks (global financial crisis, COVID-19) are visible in the data

### 7.3 Limitations and Future Research

In [ ]:
# Analyze data completeness
print("Data Limitations:")
print(f"\nMissing data by quality flag:")
quality_counts = pd.read_sql_query(
    "SELECT data_quality_flag, COUNT(*) as count FROM cleaned_data GROUP BY data_quality_flag",
    conn
)
print(quality_counts)

print(f"\nCountries with complete data:")
complete_countries = pd.read_sql_query("""
    SELECT country_name, 
           COUNT(*) as total_years,
           COUNT(CASE WHEN data_quality_flag = 'COMPLETE' THEN 1 END) as complete_years,
           ROUND(100.0 * COUNT(CASE WHEN data_quality_flag = 'COMPLETE' THEN 1 END) / COUNT(*), 1) as completeness_pct
    FROM cleaned_data
    GROUP BY country_name
    HAVING total_years >= 20
    ORDER BY completeness_pct DESC
    LIMIT 10
""", conn)
print(complete_countries)

**Limitations:**
1. Missing data for some countries and years
2. Data quality varies across countries
3. Economic indicators may not capture informal employment
4. Aggregation masks within-country regional variations
5. Causal relationships cannot be inferred from correlation alone

**Future Research Directions:**
1. Include additional economic indicators (FDI, trade, inflation)
2. Analyze sector-specific employment patterns
3. Investigate impact of specific policy interventions
4. Compare formal vs informal employment trends
5. Examine gender and age disaggregated employment data
6. Study the role of structural transformation
7. Analyze impact of external shocks (pandemics, conflicts, climate events)

### 7.4 Close Database Connection

In [ ]:
# Close database connection
conn.close()
print("Database connection closed.")
print("\n" + "="*80)
print("ANALYSIS COMPLETE!")
print("="*80)

---
## References

1. World Bank. (2024). World Development Indicators. Retrieved from https://databank.worldbank.org/
2. World Bank Open Data API. https://data.worldbank.org/
3. wbgapi Python Package. https://pypi.org/project/wbgapi/

---

## Team Contributions

Track individual contributions using Git:
```bash
git log --author="Your Name" --oneline
git shortlog -sn
```

**Project completed by:**
- [Team Member 1]: Data collection and SQL database setup
- [Team Member 2]: Data cleaning and descriptive statistics  
- [Team Member 3]: Visualizations and exploratory analysis
- [Team Member 4]: Statistical modeling and interpretation
- [Team Member 5]: Report writing and documentation

---
*End of Analysis*